In [156]:
import pandas as pd
import matplotlib.pyplot as plt
# import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import RegressorChain
from sklearn.model_selection import RandomizedSearchCV
from datetime import datetime
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import MultiOutputRegressor
# from xgboost import XGBRegressor
import numpy as np
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.layers import Input, Dense, Dropout, MultiHeadAttention, LayerNormalization
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression


In [146]:
# start_date = '2019 06 01'
# end_date = '2019 08 01'

# data_train = data.loc[start_date:end_date]
# data_test = data.loc[predict_date]
# hourly_mean = data_train.groupby(data_train.index.hour).mean()
# predictions = hourly_mean.reindex(data_train.index.hour).fillna(method='ffill')

# eval_metrics(data_test['ROI-DA'],hourly_mean['ROI-DA'])

https://scikit-learn.org/stable/modules/cross_validation.html#time-series-split

Try different parameter with selectkbest

## XGBM

In [123]:

k_best = 10  # Par exemple, sélectionner les 10 meilleures caractéristiques
selector = SelectKBest(score_func=f_regression, k=k_best)
X_train_selected = selector.fit_transform(X_train, y_train)

subsample = 1
n_estimators = 150
max_depth = 11
learning_rate = 0.0125
colsample_bytree = 0.5

XGBM = XGBM_model(X_train,y_train,learning_rate,n_estimators,max_depth,subsample,colsample_bytree)
# X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=0)
# score,XGBM = XGBM_training(X_train,y_train)

NameError: name 'XGBRegressor' is not defined

### Prediction of XGB Model

## LSTM  

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [124]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))




In [125]:

# Créer des séquences pour LSTM
def create_sequences(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 60
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, time_steps)
input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])

In [ ]:




def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.3))
    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

model = build_lstm_model(input_shape)
# Entraîner le modèle
history = model.fit(X_train_seq, y_train_seq, epochs=3, batch_size=32, validation_split=0.2)

# Sauvegarder le modèle entraîné
model.save('lstm_model.h5')

Epoch 1/3
199/199 [==============================] - 194s 949ms/step - loss: 0.0106 - val_loss: 0.0119
Epoch 2/3
199/199 [==============================] - 165s 829ms/step - loss: 0.0075 - val_loss: 0.0071
Epoch 3/3
199/199 [==============================] - 151s 757ms/step - loss: 0.0071 - val_loss: 0.0068


In [ ]:

# Préparer les données de test
data_test = df_final.loc[predict_start_date:predict_end_date]
X_test = data_test[features]
y_test = data_test['ROI-DA']

# Normaliser les données de test
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

# Créer des séquences pour LSTM sur les données de test
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, time_steps)

# Faire des prédictions
y_pred_scaled = model.predict(X_test_seq)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

# Évaluation des prédictions
y_test_actual = y_test.values[time_steps:]  # Ajuster les données réelles pour correspondre aux séquences
metrics = {'MAE': mean_absolute_error(y_test_actual, y_pred)}
print("Métriques pour la période de prédiction :", metrics)

# Exporter les résultats en format CSV
results_df = pd.DataFrame({
    'Actual ROI-DA': y_test_actual.flatten(),
    'Predicted ROI-DA': y_pred.flatten()
})

# now = datetime.now()
# timestamp = now.strftime("%Y%m%d_%H%M%S")
# filename = f'results/results_lstm_{timestamp}.csv'

# os.makedirs('results', exist_ok=True)
# results_df.to_csv(filename, index=False)

# print(f"Résultats exportés dans le fichier : {filename}")


21/21 [==============================] - 4s 148ms/step
Métriques pour la période de prédiction : {'MAE': 16.93707538119008}


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, MultiHeadAttention, LayerNormalization
from tensorflow.keras.models import Model

# Fonction pour construire le modèle Transformer
def build_transformer_model(input_shape, num_heads=2, dff=32, num_layers=2):
    inputs = Input(shape=input_shape)
    
    # Encoder
    x = inputs
    for _ in range(num_layers):
        x = MultiHeadAttention(num_heads=num_heads, key_dim=input_shape[-1])(x, x)
        x = Dropout(0.1)(x)
        x = LayerNormalization(epsilon=1e-6)(x)
        x = Dense(dff, activation='relu')(x)
    
    # Output
    outputs = Dense(1)(x)  # Output une seule valeur (ROI-DA prédit)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Exemple d'utilisation pour construire le modèle
input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])
transformer_model = build_transformer_model(input_shape)
transformer_model.compile(optimizer='adam', loss='mean_squared_error')

# Afficher la structure du modèle
transformer_model.summary()

transformer_model.fit(
    X_train_seq, y_train_seq,
    epochs=50,
    batch_size=64,
    validation_split=0.2,
    verbose=1
)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 60, 18)]     0           []                               
                                                                                                  
 multi_head_attention_4 (MultiH  (None, 60, 18)      2718        ['input_3[0][0]',                
 eadAttention)                                                    'input_3[0][0]']                
                                                                                                  
 dropout_14 (Dropout)           (None, 60, 18)       0           ['multi_head_attention_4[0][0]'] 
                                                                                                  
 layer_normalization_4 (LayerNo  (None, 60, 18)      36          ['dropout_14[0][0]']       

In [ ]:
transformer_model.fit(
    X_train_seq, y_train_seq,
    epochs=50,
    batch_size=64,
    validation_split=0.2,
    verbose=1
)

Epoch 1/50
100/100 [==============================] - 4s 29ms/step - loss: 0.0200 - val_loss: 0.0076
Epoch 2/50
100/100 [==============================] - 2s 23ms/step - loss: 0.0102 - val_loss: 0.0081
Epoch 3/50
100/100 [==============================] - 3s 27ms/step - loss: 0.0092 - val_loss: 0.0090
Epoch 4/50
100/100 [==============================] - 2s 24ms/step - loss: 0.0091 - val_loss: 0.0091
Epoch 5/50
100/100 [==============================] - 3s 26ms/step - loss: 0.0087 - val_loss: 0.0114
Epoch 6/50
100/100 [==============================] - 3s 30ms/step - loss: 0.0086 - val_loss: 0.0158
Epoch 7/50
100/100 [==============================] - 3s 25ms/step - loss: 0.0082 - val_loss: 0.0169
Epoch 8/50
100/100 [==============================] - 3s 26ms/step - loss: 0.0081 - val_loss: 0.0214
Epoch 9/50
100/100 [==============================] - 3s 26ms/step - loss: 0.0083 - val_loss: 0.0153
Epoch 10/50
100/100 [==============================] - 3s 26ms/step - loss: 0.0078 - val_lo

In [160]:
# Fonction pour construire le modèle Transformer
def build_transformer_model(input_shape, num_heads=2, dff=32, num_layers=2):
    inputs = Input(shape=input_shape)
    
    # Encoder
    x = inputs
    for _ in range(num_layers):
        attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=input_shape[-1])(x, x)
        attn_output = Dense(input_shape[-1])(attn_output)  # Projection to match input dimension
        x = LayerNormalization(epsilon=1e-6)(x + attn_output)  # Add & Norm
        ffn_output = Dense(dff, activation='relu')(x)  # Feed-forward network
        ffn_output = Dense(input_shape[-1])(ffn_output)  # Projection to match input dimension
        x = LayerNormalization(epsilon=1e-6)(x + ffn_output)  # Add & Norm

    # Output
    outputs = Dense(1)(x)  # Output une seule valeur (ROI-DA prédit)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Fonction pour créer des séquences pour LSTM/Transformer
def create_sequences(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)



In [161]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# Préparer les données de test
data_test = df_final.loc[predict_start_date:predict_end_date]
X_test = data_test[features]
y_test = data_test['ROI-DA']

X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))

time_steps = 48
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, time_steps)

# Préparer les données de test
data_test = df_final.loc[predict_start_date:predict_end_date]
X_test = data_test[features]
y_test = data_test['ROI-DA']

# Normaliser les données de test
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

# Créer des séquences pour le test
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, time_steps)

# Construction et entraînement du modèle Transformer
input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
transformer_model = build_transformer_model(input_shape)
transformer_model.compile(optimizer='adam', loss='mean_squared_error')

transformer_model.summary()
transformer_model.fit(
    X_train_seq, y_train_seq,
    epochs=3,
    batch_size=64,
    validation_split=0.2,
    verbose=1
)

# Évaluation des prédictions
y_pred_scaled = transformer_model.predict(X_test_seq)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

# Ajuster y_test pour correspondre à y_test_seq
y_test_seq_actual = y_test_scaled[time_steps:]

# Inverser la transformation pour y_test_seq_actual
y_test_seq_actual = scaler_y.inverse_transform(y_test_seq_actual)

# Évaluation des prédictions (par exemple, calculer le Mean Absolute Error)
mae = mean_absolute_error(y_test_seq_actual, y_pred)
print(f"Mean Absolute Error: {mae}")

# Affichage des prédictions et des vraies valeurs (facultatif)
plt.figure(figsize=(14, 7))
plt.plot(y_test_seq_actual, label='True ROI-DA', marker='.')
plt.plot(y_pred, label='Predicted ROI-DA', marker='.')
plt.title('True vs Predicted ROI-DA')
plt.xlabel('Time steps')
plt.ylabel('ROI-DA')
plt.legend()
plt.grid(True)
plt.show()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 48, 18)]     0           []                               
                                                                                                  
 multi_head_attention_11 (Multi  (None, 48, 18)      2718        ['input_8[0][0]',                
 HeadAttention)                                                   'input_8[0][0]']                
                                                                                                  
 dense_19 (Dense)               (None, 48, 18)       342         ['multi_head_attention_11[0][0]']
                                                                                                  
 tf.__operators__.add_6 (TFOpLa  (None, 48, 18)      0           ['input_8[0][0]',          

ValueError: Found array with dim 3. None expected <= 2.

In [ ]:

predict_start_date = pd.to_datetime(predict_start_date)
predict_end_date = pd.to_datetime(predict_end_date)
data_test = df_final.loc[predict_start_date:predict_end_date]
X_test = data_test[features]
X_test_selected = selector.transform(X_test)



y_pred = XGBM.predict(X_test)

y_true = data_test['ROI-DA']
metrics = eval_metrics(y_true, y_pred)
print("Métriques pour la période de prédiction :", metrics)

Métriques pour la période de prédiction : {'MAE': np.float64(8.457132932643809)}


In [ ]:
feature_scores = selector.scores_
feature_pvalues = selector.pvalues_

# Créer un DataFrame pour afficher les résultats
feature_names = X_train.columns
scores_df = pd.DataFrame({'Feature': feature_names, 'Score': feature_scores, 'p-Value': feature_pvalues})

# Trier le DataFrame par score décroissant
scores_df = scores_df.sort_values(by='Score', ascending=False)
print(scores_df)

           Feature        Score        p-Value
2   ROI-DA_lag_48h  2326.000460   0.000000e+00
5   ROI-DA_lag_49h  1748.572402   0.000000e+00
8   ROI-DA_lag_50h   948.543423  6.401506e-197
15            hour   893.632993  3.224883e-186
1      Gas_lag_48h   463.534010  5.469151e-100
4      Gas_lag_49h   463.311213  6.077771e-100
7      Gas_lag_50h   462.972184  7.136376e-100
10     Gas_lag_51h   462.256921   1.001397e-99
13     Gas_lag_52h   461.311220   1.567313e-99
11  ROI-DA_lag_51h   428.331642   9.861167e-93
0      Oil_lag_48h   339.213665   3.210587e-74
3      Oil_lag_49h   338.644120   4.222528e-74
6      Oil_lag_50h   337.771573   6.425096e-74
9      Oil_lag_51h   336.734115   1.058448e-73
12     Oil_lag_52h   335.603186   1.823999e-73
17     day_of_year   215.146720   4.355626e-48
14  ROI-DA_lag_52h   157.204057   1.000974e-35
16     day_of_week   126.194107   4.602436e-29


In [ ]:
import os
# Ajouter les métriques au dataframe
metrics_df = pd.DataFrame([metrics])
print(metrics_df)

# Générer le nom de fichier avec la date et l'heure actuelles
now = datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")
filename = f'results/results_{timestamp}.csv'

# Créer le répertoire 'results' s'il n'existe pas
os.makedirs('results', exist_ok=True)

# Exporter les scores, les p-values et les métriques MAE dans un fichier CSV
full_results_df = pd.concat([scores_df, metrics_df], axis=1)
# full_results_df.to_csv(filename, index=False)

print(f"Résultats exportés dans le fichier : {filename}")

        MAE
0  8.457133
Résultats exportés dans le fichier : results/results_20240704_110948.csv


In [ ]:
prediction_results = pd.DataFrame(index=data_test.index)
prediction_results['ROI-DA réel'] = y_true.values
prediction_results['ROI-DA prédit'] = y_pred
prediction_results.loc['2020 04 01':'2020 04 28'].plot()
plt.legend()
plt.ylabel('euro(€)')
plt.xlabel('days')
plt.title('ROI-DA prediction')

NameError: name 'y_true' is not defined

In [ ]:
y_true

NameError: name 'y_true' is not defined

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

start_date = '2019-06-01'
end_date = '2020-04-07'
predict_start_date = '2020-04-08'
predict_end_date = '2020-04-14'  # Prédiction pour la semaine suivante
number_days = 7
results = []

# Initialiser les dates
current_date = pd.to_datetime(start_date) + pd.Timedelta(days=number_days)
end_date = pd.to_datetime(end_date)
predict_start_date = pd.to_datetime(predict_start_date)
predict_end_date = pd.to_datetime(predict_end_date)
# Fonction d'évaluation des métriques
def eval_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    return {'MAE': mae}

params = {
    'objective': 'reg:squarederror', 
    'max_depth': 6,                   
    'eta': 0.05,                       
    'eval_metric': 'rmse',             
    'subsample': 0.8,                 
    'colsample_bytree': 0.8,           
    'gamma': 0                         
}
model = xgb.XGBRegressor(**params)


# Filtrer les données pour l'entraînement sur 10 mois
train_start_date = pd.to_datetime(start_date)
train_end_date = end_date - pd.Timedelta(days=number_days)
data_train = df_final.loc[train_start_date:train_end_date]

# # Calcul des moyennes horaires sur les 7 jours précédents pour ROI-DA
# hourly_mean = data_train['ROI-DA'].groupby(data_train.index.hour).mean()
# df_final['7_day_hourly_mean'] = df_final.index.map(hourly_mean)

# Préparer les données pour l'entraînement
X_train = data_train.iloc[:-48][['Oil', 'Gas','ROI-DA']]
y_train = data_train.iloc[-48:]['ROI-DA']  # Les 48 dernières valeurs de ROI-DA pour prédire les deux prochains jours 

# Conversion en DMatrix pour XGBoost
model.fit(X_train, y_train)
# Définir les paramètres du modèle
num_round = 1000
# bst = xgb.train(params, dtrain, num_round)

# Préparer les données de test pour la semaine suivante à partir du 8 avril 2020
data_test = df_final.loc[predict_start_date:predict_end_date]
X_test = data_test.iloc[:-48][['Oil', 'Gas', 'ROI-DA']]

# Conversion en DMatrix pour XGBoost
y_pred = model.predict(X_test)

# # Faire des prédictions
# y_pred = bst.predict(dtest)

# Évaluation des prédictions (optionnel, pour vérification)
y_true = data_test.iloc[-48:]['ROI-DA']
metrics = eval_metrics(y_true, y_pred)
print("Metrics for prediction period:", metrics)


XGBoostError: [11:09:49] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\data\data.cc:508: Check failed: this->labels.Size() % this->num_row_ == 0 (48 vs. 0) : Incorrect size for labels.

In [ ]:
import pandas as pd
import xgboost as xgb

# Charger et préparer les données
# Exemple de chargement de données fictives


# Définir les dates
start_date = '2019-06-01'
end_date = '2020-04-07'
number_days = 7

# Filtrer les données pour l'entraînement sur 10 mois
train_start_date = pd.to_datetime(start_date)
train_end_date = pd.to_datetime(end_date) - pd.Timedelta(days=number_days)
data_train = df_final.loc[train_start_date:train_end_date]

# Préparer les données pour l'entraînement
X_train = []
y_train = []

# Création des échantillons d'entraînement et des cibles de prédiction
for i in range(len(data_train) - 48):
    X_window = data_train[['Oil', 'Gas', 'ROI-DA']].iloc[i:i+48]  # Caractéristiques sur les 48 dernières heures
    y_window = data_train['ROI-DA'].iloc[i+48:i+96]              # ROI-DA pour les 48 prochaines heures
    X_train.append(X_window)
    y_train.append(y_window)

# Concaténer les listes en DataFrame
X_train = pd.concat(X_train)
y_train = pd.concat(y_train).values  # Convertir en un tableau NumPy

# Conversion en DMatrix pour XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)

# Définir les paramètres du modèle
params = {
    'objective': 'reg:squarederror',  # Pour un problème de régression
    'max_depth': 6,                    # Profondeur des arbres plus grande
    'eta': 0.05,                       # Taux d'apprentissage plus faible
    'eval_metric': 'rmse',             # Erreur quadratique moyenne racine
    'subsample': 0.8,                  # Sous-échantillonnage des échantillons
    'colsample_bytree': 0.8,           # Sous-échantillonnage des features
    'gamma': 0                         # Pas de réduction minimale de la perte
}
num_round = 1000
bst = xgb.train(params, dtrain, num_round)

# Prédiction des prochaines 48 heures
# Utilisation des données les plus récentes disponibles pour faire la prédiction
X_recent = data_train[['Oil', 'Gas', 'ROI-DA']].tail(48)  # Sélectionner les 48 dernières lignes de X_train

# Conversion en DMatrix pour XGBoost
drecent = xgb.DMatrix(X_recent)

# Faire des prédictions pour les 48 prochaines heures
y_pred = bst.predict(drecent)

# Afficher les prédictions
print("Predicted ROI-DA for the next 48 hours:")
print(y_pred)


XGBoostError: [11:29:43] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\data\data.cc:508: Check failed: this->labels.Size() % this->num_row_ == 0 (346824 vs. 0) : Incorrect size for labels.

In [ ]:
results_df = pd.DataFrame(results)
results_df

,start_date,end_date,metrics
0,2019-06-08,2019-06-14,{'MAE': 0.12910339565934797}
1,2019-06-15,2019-06-21,{'MAE': 0.12649279574690175}
2,2019-06-22,2019-06-28,{'MAE': 0.14494916139800004}
3,2019-06-29,2019-07-05,{'MAE': 0.11951590676143278}
4,2019-07-06,2019-07-12,{'MAE': 0.13356749225484943}
5,2019-07-13,2019-07-19,{'MAE': 0.16253624435951003}
6,2019-07-20,2019-07-26,{'MAE': 0.14154209429642214}
7,2019-07-27,2019-08-02,{'MAE': 0.15638298718682644}
8,2019-08-03,2019-08-09,{'MAE': 0.12607029197955935}
9,2019-08-10,2019-08-16,{'MAE': 0.16883613218110194}


In [ ]:
# Calculer la moyenne des MAE
mean_mae = results_df['metrics'].mean()
print(f"Mean MAE over the year: {mean_mae}")

# Afficher les résultats dans un plot
plt.figure(figsize=(12, 6))
plt.plot(results_df['date'], results_df['metrics'], label='Daily MAE')
plt.axhline(y=mean_mae, color='r', linestyle='--', label=f'Mean MAE: {mean_mae:.2f}')
plt.xlabel('Date')
plt.ylabel('Mean Absolute Error (MAE)')
plt.title('Daily MAE Over One Year')
plt.legend()
plt.grid(True)
plt.show()

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [ ]:
def train_test_split_tbs(final_data):
    train,test = final_data.iloc[:int(0.8*final_data.shape[0]), :] , final_data.iloc[int(0.8*final_data.shape[0]):, :]
    print(train.shape)
    print(test.shape)
    
    return train,test

datatrain,datatest = train_test_split_tbs(data)

(7027, 1)
(1757, 1)


In [ ]:
correlation_matrix = data.corr()

# Afficher le tableau de corrélation avec la colonne ROI-DA
correlation_with_ROI_DA = correlation_matrix['ROI-DA']

print("Tableau de corrélation avec la colonne ROI-DA :")
print(correlation_with_ROI_DA)

Tableau de corrélation avec la colonne ROI-DA :
ROI-DA    1.0
Name: ROI-DA, dtype: float64


In [ ]:
# def normalize(x_train,x_test):
#     from sklearn.preprocessing import MinMaxScaler
#     scaler = MinMaxScaler().fit(x_train)
#     x_train_norm = scaler.transform(x_train)
#     x_test_norm = scaler.transform(x_test)
#     return x_train_norm,x_test_norm

# datatrain,datatest = normalize(datatrain,datatest)

In [ ]:
# datatrain